# Train

TODO convert this to train.py

In [1]:
import os
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter

from data_loader.cityscapes import CityscapesDataLoader
from model.deeplabv3plus import DeepLabv3Plus

from model.metric import SegmentationMetrics

## Checkpoints and Tensorboard

In [2]:
run_dir = "run3"
checkpoint_file = os.path.join("./experiments", "checkpoints", run_dir, "best_model.pt")
log_dir = os.path.join("./experiments", 'runs', run_dir)
writer = SummaryWriter(log_dir=log_dir)

## Get DataLoader

In [3]:
img_root = "./data/leftImg8bit"
mask_root = "./data/gtFine"
batch_size = 4

cityscapes = CityscapesDataLoader(img_root, mask_root, batch_size)

In [8]:
len(cityscapes.train_set) / 4

743.75

In [9]:
90000/744

120.96774193548387

## Initialize Model, Loss, and Optimizer

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = cityscapes.train_set.num_classes  # 0 -> 19
ignore_idx = cityscapes.train_set.ignoreId  # ignore 19
learning_rate = 0.001
momentum = 0.9
weight_decay = 0.0005

model = DeepLabv3Plus(num_classes).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=ignore_idx)
optimizer = torch.optim.SGD(
    model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay
)

In [ ]:
def save_checkpoint(epoch, model, optimizer, path):
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        path,
    )


def train_one_epoch(epoch, model, criterion, optimizer, data_loader, device):
    model.train()

    total_loss = 0
    n_batches = len(data_loader)
    for i, sample in enumerate(tqdm(data_loader)):
        images = sample["image"].to(device)
        masks = sample["mask"].to(device)

        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        writer.add_scalar("train_iter_loss", loss.item(), i + n_batches * epoch)

    avg_loss = total_loss / n_batches
    writer.add_scalar("train_epoch_avg_loss", avg_loss, epoch)
    return avg_loss


def validate(epoch, model, criterion, data_loader, num_classes, device):
    model.eval()

    total_loss = 0
    n_batches = len(data_loader)
    metrics = SegmentationMetrics(num_classes=num_classes, ignore_idx=ignore_idx)
    with torch.no_grad():
        for i, sample in enumerate(tqdm(data_loader)):
            images = sample["image"].to(device)
            masks = sample["mask"].to(device)

            pred = model(images)
            loss = criterion(pred, masks)
            total_loss += loss.item()

            pred_cls = torch.argmax(pred, dim=1)
            metrics.update(pred_cls, masks)

    model.train()

    avg_loss = total_loss / n_batches
    ious, mIoU = metrics.iou()
    writer.add_scalar("val_epoch_avg_loss", avg_loss, epoch)
    writer.add_scalar("val_epoch_mIoU", mIoU, epoch)
    return avg_loss, ious, mIoU

In [ ]:
patience = 15
best_mIoU = 0
best_epoch = 0
for epoch in range(1000):
    train_loss = train_one_epoch(
        epoch, model, criterion, optimizer, train_loader, device
    )
    val_loss, ious, mIoU = validate(
        epoch, model, criterion, val_loader, num_classes, device
    )
    print(f"Epoch {epoch}: val_loss {val_loss:.4f} | mIoU {mIoU: .4f}")

    if mIoU > best_mIoU:
        best_mIoU = mIoU
        best_epoch = epoch
        save_checkpoint(epoch, model, optimizer, checkpoint_file)
    elif epoch - best_epoch >= patience:
        print(f"Early Stopping at epoch {epoch}")
        break

In [ ]:
writer.close()